In [1]:
import urllib
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup


url_wikipedia = 'https://ru.wikipedia.org'
url_es = 'https://ru.wikipedia.org/wiki/Категория:Города_по_субъектам_Российской_Федерации'
url_es_q  = urllib.parse.quote(url_es, safe='/:')
print(url_es_q)

https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0_%D0%BF%D0%BE_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%B0%D0%BC_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8


In [2]:
es_bs = BeautifulSoup(urllib.request.urlopen(url_es_q), 'lxml')

In [3]:
all_es = []
for i in range(30):
    # load links:
    bef = len(all_es)
    for links in es_bs.find_all('div', class_='mw-category-group'):
        for l in links.find_all('a'):
            all_es.append((l['title'], url_wikipedia+l['href']))

In [4]:
all_es

[('Категория:Категории городов по субъектам Российской Федерации',
  'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_%D0%BF%D0%BE_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%B0%D0%BC_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8'),
 ('Категория:Города Крыма',
  'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0_%D0%9A%D1%80%D1%8B%D0%BC%D0%B0'),
 ('Категория:Категории по городам субъектов Российской Федерации',
  'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8_%D0%BF%D0%BE_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%B0%D0%BC_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%

In [5]:
import pandas as pd
all_es_prop = []

for i, (n, lnk) in enumerate(all_es[:9]):
    es = BeautifulSoup(urllib.request.urlopen(lnk), 'lxml')
    es_pr = {}
    city_info = {}

    all_cities_in_group = []
    mw_pages = es.find("div", {"id": "mw-pages"})
    group_cities_links = []
    saved_th = []
    cities_rows = []

    cities = []

    df_parsed = pd.DataFrame()

    if mw_pages is not None:
        group_cities = mw_pages.find_all("a")
        for each_a in group_cities:
            if "Городские" in each_a["title"]:
                group_cities_links.append(f"https://ru.wikipedia.org{each_a['href']}")
                all_cities_in_group.append(each_a["title"])
                #print(each_a["href"], each_a["title"])
        #print(group_cities)
        #all_cities_in_group.append(group_cities["title"])
        #print(f"https://ru.wikipedia.org{group_cities['href']}")
        #group_cities_links.append(f"https://ru.wikipedia.org{group_cities['href']}")

    for link in group_cities_links:
        cs = BeautifulSoup(urllib.request.urlopen(link), 'lxml')
        cities_table = cs.find("table", class_="standard")
        #print(link)
        if cities_table is not None:
            for row in cities_table.find_all("tr"):
                th = row.find_all("th")
                #print("123")
                #print(th.text)
                if th:
                    #pass
                    #print(f"Th length: {len(th)}")
                    for i in range(len(th)):

                        #print(th[i].text.strip(), end=' || ')
                        saved_th.append(th[i].text.strip())
                        #df_parsed[th[i].text.strip()] = 0
                td = row.find_all('td')
                #print(td.text)
                if td:
                    #print("123")
                    for i in range(len(td)):
                        a_img = td[i].find_all("a", class_="image")

                        if a_img:
                            for each_img in a_img:
                                img_src = each_img.find_all("img")
                                if img_src:
                                    for each_src in img_src:

                                        image_link = each_src["src"]

                                        # print(f"Th[i]: {i}")
                                        # print(f"Th now: {saved_th}")
                                        # print(f"Th length now: {len(saved_th)}")
                                        # print(f"Image link in: {saved_th[i]}")

                        #city_info[saved_th[i]] = td[i].text.strip()
                        #es_pr[td[i].text.strip() if saved_th[i] == "№" else len(es_pr)] = {saved_th[i]: td[i].text.strip()}

                        es_pr[saved_th[i]] = td[i].text.strip()
                        if i == len(td)- 1:
                            #print(es_pr)
                            cities.append(es_pr)
                            #print(pd.json_normalize(es_pr))

                            es_pr = {}
                        #df_parsed[saved_th[i]] = td[i].text.strip()
                        #print(td[i].text.strip(), "----", saved_th[i])
                    #es_pr[td[i].text.strip()] = city_info
                    #print(cities)
                    #print("123")
    if len(cities) > 0:
        print(cities)
        #print(es_pr)
        all_es_prop.append((n, {"Города": cities}))
        df_parsed[n] = cities
        #print('{}: {}, Название города: {}'.format(i, n, city_name))


[{'№': '1', 'название': 'Алупка', 'район[5] / город   республиканского значения[6]': 'Ялта', 'население(чел.)': '↗9063[4]', 'основание / первое упоминание': '960', 'статус города': '1938', 'герб': ''}, {'№': '2', 'название': 'Алушта', 'район[5] / город   республиканского значения[6]': 'Алушта', 'население(чел.)': '↗31\xa0364[4]', 'основание / первое упоминание': 'VI в.', 'статус города': '1902', 'герб': ''}, {'№': '3', 'название': 'Армянск', 'район[5] / город   республиканского значения[6]': 'Армянск', 'население(чел.)': '↘20\xa0692[4]', 'основание / первое упоминание': '1736', 'статус города': '1993', 'герб': ''}, {'№': '4', 'название': 'Бахчисарай', 'район[5] / город   республиканского значения[6]': 'Бахчисарайский район', 'население(чел.)': '↗28\xa0609[4]', 'основание / первое упоминание': '1389', 'статус города': '', 'герб': ''}, {'№': '5', 'название': 'Белогорск', 'район[5] / город   республиканского значения[6]': 'Белогорский район', 'население(чел.)': '↗17\xa0445[4]', 'основание

In [6]:
pd.json_normalize(all_es_prop[0][1]["Города"])

,№,название,район[5] / город республиканского значения[6],население(чел.),основание / первое упоминание,статус города,герб
0,1,Алупка,Ялта,↗9063[4],960,1938,
1,2,Алушта,Алушта,↗31 364[4],VI в.,1902,
2,3,Армянск,Армянск,↘20 692[4],1736,1993,
3,4,Бахчисарай,Бахчисарайский район,↗28 609[4],1389,,
4,5,Белогорск,Белогорский район,↗17 445[4],XIII в.,,
5,6,Джанкой,Джанкой,↘37 014[4],1855,1917,
6,7,Евпатория,Евпатория,↘107 877[4],497 до н. э.,,
7,8,Керчь,Керчь,↗154 621[4],VII в. до н. э.,,
8,9,Красноперекопск,Красноперекопск,↗25 569[4],1932,1966,
9,10,Саки,Саки,↘24 285[4],,1952,
